In [1]:
import pandas as pd
import geopandas as gpd
import fiona
import matplotlib.pyplot as plt
import numpy as np
import contextily as ctx
import rasterio
from rasterio.crs import CRS
from rasterio.transform import from_bounds
from shapely import force_2d
from shapely.geometry import box, Point, Polygon, MultiPolygon
from scipy.spatial import cKDTree
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel, ConstantKernel
from flood_density.plots import city_bounds_and_density_plot
from flood_density.preprocess import get_bounds_xy_min_max, extract_bounds_polygon, polygon_to_geodataframe, extract_city_data, coordinates_to_box,points_geocoordinates,convert_points_in_gdf, extract_city_bounds_from_df_to_gdf , gdf_to_geojson, clip_density_to_urban_area

from typing import List, Tuple

CRS_4326 = 4326


# Etapa 1 : Obtener datos .kml de la ciudad de La Plata.

In [2]:
def convert_kml_to_gdf(kml_file: str) -> gpd.GeoDataFrame:
    gdf = gpd.read_file(kml_file, driver="KML")
    return gdf

In [3]:
gdf_from_kml = convert_kml_to_gdf('laplata_cascourbano.kml')

/home/andy/.local/lib/python3.10/site-packages/pyogrio/geopandas.py:275: UserWarning: More than one layer found in 'laplata_cascourbano.kml': 'La Plata' (default), 'La Plata (#2)', 'La Plata (#3)'. Specify layer parameter to avoid this warning.
  result = read_func(


In [ ]:
#def export_to_geojson(gdf: gpd.GeoDataFrame, output_path: str) -> gpd.GeoDataFrame:
    #gdf.to_file(output_path, driver='GeoJSON')
    #return gdf

In [ ]:
#gdf_peligrosidad = export_to_geojson(gdf_from_kml, 'laplata_cascourbano.geojson')

In [ ]:
#def plot_gdf(gdf: gpd.GeoDataFrame):
    return gdf.plot()

In [ ]:
#plot_gdf(gdf_peligrosidad)

# Etapa 2: Obtener las coordenadas de los puntos de la ciudad, formamos un polígono y luego, transformamos a un gdf.


In [ ]:
#¿Cómo obtengo bounds_dict? son coordenadas de la ciudad de La Plata Desde el archivo geojson

In [ ]:
# Cargar el archivo GeoJSON del plot
gdf_la_plata = gpd.read_file("salida.geojson")

In [ ]:
#Extraemos las coordenadas geográficas de la ciudad 
city_bounds_coordinates= get_bounds_xy_min_max(gdf_la_plata)
city_bounds_coordinates

In [ ]:
# Convertimos las coordenadas de la ciudad a un box
boxx = coordinates_to_box(city_bounds_coordinates)

In [ ]:
#Usar el dataframe del polígono que forma las coordenadas de la LP
gdf_la_plata_from_polygon = gpd.GeoDataFrame({'geometry': [boxx]}, crs='EPSG:4326')

In [ ]:
# Convertir a EPSG 32721 (UTM zona 21S)
gdf_la_plata_from_polygon = gdf_la_plata_from_polygon.to_crs(epsg=32721)

# Etapa 3 : Obtener y transformar datos de densidad poblacional de la ciudad de La Plata.

Primero extraemos las filas que pertenecen a la ciudad de La Plata(sigue siendo un .csv), luego convertimos ese dataframe en un geodataframe.

In [ ]:
#Levantamos el dataset completo
df_arg = pd.read_csv("arg_pd_2020_1km_ASCII_XYZ.csv") 

In [ ]:
# Extraemos las filas referidas a la ciudad de La Plata
df_lp_coordinates = extract_city_data(df_arg,'la_plata',city_bounds_coordinates)


In [ ]:
#Leemos los datos específicos de la ciudad de La Plata
df_lp_coordinates = pd.read_csv('la_plata_population_2020.csv')


In [ ]:
#Convertir dataframe de los puntos pertenecientes a la ciudad de La Plata en un geodataframe
gdf_lp_coordinates = extract_city_bounds_from_df_to_gdf(df_lp_coordinates, 'Y','X')

In [ ]:
# Extraemos las coordenadas de los puntos dentro de la ciudad de La Plata
casco_urbano = clip_density_to_urban_area(gdf_lp_coordinates,gdf_la_plata)

In [ ]:
# Función para forzar geometrías 2D
#def to_2d(geom: gpd.GeoSeries) -> gpd.GeoSeries:
#    if geom.has_z:
#        return type(geom)([(x, y) for x, y, *rest in geom.coords])
#    return geom

In [ ]:
# Reproyectar a EPSG 32721 (UTM zona 21S)
casco_urbano_utm = casco_urbano.to_crs(epsg=32721)

In [ ]:
# Calcular centroides en el CRS proyectado
centroides = casco_urbano_utm.geometry.centroid

In [ ]:
# Convertir centroides a un array de coordenadas
coords_centroides = np.array([[pt.x, pt.y] for pt in centroides])

In [ ]:
# Extraer densidades de casco_urbano_utm
values_density_lp = casco_urbano_utm['Z'].values

In [ ]:
#Extraer los límites del polígono de la ciudad de La Plata
bounds_lp = gdf_la_plata_from_polygon.total_bounds  # [minx, miny, maxx, maxy]
bounds_lp

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
sc = ax.scatter(coords_centroides[:, 0], coords_centroides[:, 1],
                c=values_density_lp, s=100,
                cmap='viridis', alpha=0.9,
                edgecolor='black', linewidth=1,
                marker='o')

plt.colorbar(sc, ax=ax, label='Densidad')
ax.set_title("Centroides (EPSG:32721)")
plt.show()

# Etapa 6 : Merge polígono y zona de densidad de la ciudad de La Plata en un plot.

In [ ]:
city_bounds_and_density_plot(gdf_la_plata_from_polygon, coords_centroides, values_density_lp, bounds_lp)

In [ ]:
# Visualización de la densidad poblacional de La Plata usando kriging

# Extraer coordenadas de los centroides,y valores desde el GeoDataFrame
coords =np.array([[point.x, point.y] for point in casco_utm.geometry.centroid]) 
values_density = casco_urbano['densidad_poblacional_lp'].values

print(f"Puntos de datos: {len(coords)}")    
print(f"Rango de valores: {values_density.min():.2f} - {values_density.max():.2f}")

# Crear grid de puntos para interpolación
grid_x, grid_y = np.mgrid[bounds_proj[0]:bounds_proj[2]:150j, bounds_proj[1]:bounds_proj[3]:150j]
grid_coords = np.column_stack([grid_x.ravel(), grid_y.ravel()])

#Definir kernel para el modelo de kriging
kernel = (ConstantKernel(1.0) * RBF(length_scale=1000.0) + 
          WhiteKernel(noise_level=0.1, noise_level_bounds=(1e-10, 1e3)))

# Crear modelo de kriging
gpr = GaussianProcessRegressor(kernel=kernel,
                                n_restarts_optimizer=10,
                                alpha=1e-04,
                                normalize_y=True)

# Entrenar el modelo
gpr.fit(coords, values_density)

#Predecir en el grid
grid_2_kriging, pred_std = gpr.predict(grid_coords, return_std=True)
grid_z = grid_2_kriging.reshape(grid_x.shape)
grid_uncertainty = pred_std.reshape(grid_x.shape)

# Visualización de la superficie Kriging interpolada
fig, ax = plt.subplots(figsize=(10, 10))

# Plotear superficie Kriging interpolada
contour = ax.contourf(grid_x, grid_y, grid_z, levels=30, cmap='viridis', alpha=0.5)

# Plotear los polígonos originales con bordes
casco_utm.plot(column='densidad_poblacional_lp', 
               cmap='viridis',
               alpha=0.5,
               edgecolor='black',
               linewidth=1.0,
               ax=ax)

# Plotear los puntos centroides
scatter = ax.scatter(coords[:, 0], coords[:, 1],
                    c=values_density,
                    cmap='viridis',
                    s=30,
                    edgecolors='black',
                    linewidths=1,
                    zorder=5)

# Ajustar límites del gráfico según los límites de La Plata
ax.set_xlim(bounds_proj[0], bounds_proj[2])
ax.set_ylim(bounds_proj[1], bounds_proj[3])

#Mapa base más sutil
ctx.add_basemap(ax, crs=casco_utm.crs.to_string(),
               source=ctx.providers.CartoDB.Positron,
               alpha = 0.9)

# Agregar colorbar para el scatter
plt.colorbar(scatter, ax=ax, label='Densidad poblacional',shrink=0.7, aspect=25)

# Agregar título y etiquetas
ax.set_title('Interpolación Kriging - Densidad Poblacional La Plata')
ax.set_xlabel('X (UTM)')
ax.set_ylabel('Y (UTM)')

plt.tight_layout()
plt.show()

In [ ]:
def export_to_geotiff(grid_data: np.ndarray, gdf: gpd.GeoDataFrame, filename: str, crs_epsg: int = 3857):

     # Crear un GeoDataFrame con los límites del área de La Plata en EPSG:4326
    gdf_bounds_4326 = gpd.GeoDataFrame(geometry=[bbox], crs="EPSG:4326")

    # Transformar al CRS del plot
    gdf_bounds_utm = gdf_bounds_4326.to_crs(casco_utm.crs)

    # Obtener límites en coordenadas proyectadas
    bounds_proj = gdf_bounds_utm.total_bounds  # [xmin, ymin, xmax, ymax]
    height, width = grid_data.shape

    # Calcula la transformación georreferenciada
    transform = from_bounds(bounds_proj[0], bounds_proj[1], bounds_proj[2], bounds_proj[3], width, height)

    with rasterio.open(
        filename,
        'w',
        driver='GTiff',
        height=height,
        width=width,
        count=1,
        dtype=grid_data.dtype,
        crs=CRS.from_epsg(crs_epsg),
        transform=transform,
        compress='lzw'
    ) as dst:
        dst.write(grid_data, 1)
        dst.set_band_description(1, 'Densidad Poblacional Kriging')



In [ ]:
# Exportar superficie interpolada
export_to_geotiff(grid_z, gdf_bounds_4326, 'kriging_densidad_poblacional.tif', 3857)

# Etapa 5 : superposición de mapas de densidad y peligrosidad de inundación de la ciudad de La Plata (QGIS).